# Project Data Sample Review 

In [1]:
import pandas as pd
import numpy as np
import os
from collections import Counter
%matplotlib inline
pd.set_option('max_rows',300)

In [4]:
projects=pd.read_csv('../../Data/EWS_Published Project_Listing_DD.csv', encoding='ISO-8859-1')
projects = projects[projects['EWS ID'].notnull()]

In [5]:
projects.shape

(6847, 62)

** Null Check **

In [6]:
projects.count()/len(projects.index)

EWS ID                   1.000000
ProjectNumber            0.998832
Published                0.998832
Bank Risk Rating         0.998832
Project Status           0.936469
EWS URL                  0.998832
Detailed Analysis URL    0.000000
Project Name             0.998686
City                     0.169563
Country Count            0.998832
Country 1                0.908719
Country 2                0.035782
Country 3                0.019863
Country 4                0.011830
Country 5                0.007741
Country 6                0.005112
Country 7                0.003359
Country 8                0.001899
Country 9                0.001607
Country 10               0.001460
Country 11               0.001022
Country 12               0.000584
Borrower or Client       0.760917
Private Actor Count      0.997955
Private Actor 1          0.077114
Private Actor 2          0.017964
Private Actor 3          0.006572
Private Actor 4          0.003067
Private Actor 5          0.002191
Private Actor 

## Project Description Column will Likely Be most Useful for Matching 

**Notes**
    * Some descriptions are pretty short - not sure how easy it will be to match to those
    * Some of the other fields will likely be useful (Country, Borrower or Client, etc.)

In [7]:
# for i in projects.sample(15)['Project Description']:
#     print(i)
#     print('*****\n')

## Looking at the Sector Data 

This could be a dataset that could help in the tagging of the news articles with (Sector Infortmation)

In [8]:
def get_category_cols(category, additional_removes=None):
    cols = [i for i in projects.columns if category in i]
    cols.remove(category + ' Count')
    if additional_removes:
        [cols.remove(i) for i in additional_removes]
    return cols

sector_cols = get_category_cols('Sector')   
all_sectors = projects[sector_cols].as_matrix().flatten()


In [9]:
for i in Counter(all_sectors): print(i)

Agriculture and Forestry
nan
Infrastructure
Transport
Construction
Humanitarian Response
Industry and Trade
Water and Sanitation
Education and Health
Law and Government
Energy
Hydropower
Climate and Environment
Finance
Mining
Communications
Technical Cooperation


# NOTE 

Might be able to use this to also classify the Bank and Country 

**Countries**

In [10]:
country_cols = get_category_cols('Country')
all_countries = projects[country_cols].as_matrix().flatten()
country_counter = Counter(all_countries)
print(len(country_counter))
country_counter

180


Counter({'Afghanistan': 48,
         'Albania': 27,
         'Algeria': 3,
         'Angola': 8,
         'Argentina': 163,
         'Armenia': 50,
         'Austria': 24,
         'Azerbaijan': 31,
         'Bahamas': 30,
         'Bangladesh': 140,
         'Barbados': 14,
         'Belarus': 17,
         'Belgium': 25,
         'Belize': 17,
         'Benin': 13,
         'Bhutan': 36,
         'Bolivia': 106,
         'Bosnia and Herzegovina': 34,
         'Botswana': 5,
         'Brazil': 213,
         'Bulgaria': 22,
         'Burkina Faso': 23,
         'Burundi': 9,
         'Cambodia': 72,
         'Cameroon': 26,
         'Canada': 1,
         'Cape Verde': 6,
         'Central African Republic': 16,
         'Chad': 17,
         'Chile': 82,
         'China': 255,
         'Colombia': 168,
         'Comoros': 2,
         'Congo, Democratic Republic of': 26,
         'Congo, Republic of': 9,
         'Cook Islands': 1,
         'Costa Rica': 49,
         'Croatia': 37,
      

**Banks**

In [11]:
banks_cols = get_category_cols('Bank', ['Bank Risk Rating'])
all_banks = projects[banks_cols].as_matrix().flatten()
all_banks = [i for i  in all_banks if  pd.notnull(i)] ## Something weird with the nulls in this one. 
banks_counter = Counter(all_banks)
print(len(banks_counter))
banks_counter

13


Counter({'African Development Bank (AFDB)': 86,
         'Asian Development Bank (ADB)': 1159,
         'Asian Infrastructure Investment Bank (AIIB)': 62,
         'European Bank for Reconstruction and Development (EBRD)': 496,
         'European Investment Bank (EIB)': 1029,
         'Green Climate Fund (GCF)': 72,
         'Inter-American Development Bank (IADB)': 1585,
         'Inter-American Investment Corporation (IIC)': 135,
         'International Finance Corporation (IFC)': 1057,
         'Multilateral Investment Guarantee Agency (MIGA)': 88,
         'Netherlands Development Finance Company (FMO)': 249,
         'New Development Bank (NDB)': 13,
         'World Bank (WB)': 888})

In [12]:
for b in banks_counter: print(b)

World Bank (WB)
Inter-American Investment Corporation (IIC)
African Development Bank (AFDB)
New Development Bank (NDB)
Asian Infrastructure Investment Bank (AIIB)
Green Climate Fund (GCF)
Multilateral Investment Guarantee Agency (MIGA)
Inter-American Development Bank (IADB)
European Bank for Reconstruction and Development (EBRD)
European Investment Bank (EIB)
Netherlands Development Finance Company (FMO)
Asian Development Bank (ADB)
International Finance Corporation (IFC)


------------

# Compare to the Labeled Data 

In [14]:
feedly = pd.read_pickle('../../Data/Feedly_Processed_DF_cleaned.pkl')

In [18]:
labels = pd.read_excel('../../Data/SetupData/Feedly_Labeled_Data.xlsx')

In [19]:
labels = labels[labels.article_id.isin(feedly.article_id)]
print(labels.shape)

(433, 16)


-----------

## Projects 

**Valid EWS HyperLink**

We have some repeat projects 

In [20]:
labels[(labels['EWS hyperlink'].notnull()) & (labels['EWS hyperlink'].str.contains('//ews')) ]['EWS hyperlink'].value_counts().head()

https://ewsdata.rightsindevelopment.org/projects/20150676-tanap-trans-anatolian-natural-gas-pipeline/                                                                                                                                                                                                 16
https://ewsdata.rightsindevelopment.org/projects/50410-001-asean-distributed-power-project-initial-poverty-a/                                                                                                                                                                                          6
https://ewsdata.rightsindevelopment.org/projects/00057-bangladesh-bhola-ipp/                                                                                                                                                                                                                           6
https://ewsdata.rightsindevelopment.org/projects/000015-bangladesh-natural-gas-infrastructure-and-efficie/; h

We have this Many Unique Projects that were matched to Articles 

In [21]:
labels[(labels['EWS hyperlink'].notnull()) & (labels['EWS hyperlink'].str.contains('//ews')) ]['EWS hyperlink'].nunique()

41

In [22]:
project_labeled_data = labels.copy(deep=True)
project_labeled_data = project_labeled_data[project_labeled_data.Matched.notnull()]
print(project_labeled_data.shape)

(112, 16)


### Clean the ID Columns for unmatched Projects 

In [23]:
project_labeled_data.loc[project_labeled_data.Matched == 0,'Projects(EWSProjectID)'] = np.nan
project_labeled_data.loc[project_labeled_data.Matched == 0,'EWS Project Name'] = np.nan
project_labeled_data.loc[project_labeled_data.Matched == 0,'EWS hyperlink'] = np.nan



**Process to Check How Many We can Match ** 

In [24]:
project_labels = project_labeled_data[project_labeled_data.Matched ==1]
project_labels = project_labels[project_labels['EWS hyperlink'].str.contains('https://ews')]

In [25]:
project_ids = []
for i in project_labels['Projects(EWSProjectID)']:
    project_ids += [j.strip() for j in  i.split(',')]

In [26]:
project_ids = list(set(project_ids))
print(len(project_ids))

46


In [27]:
print(len(set(project_ids) & set(projects.ProjectNumber.unique())), 'are matched')

46 are matched


-----------

# Sector 

```Communications
Construction
Hydropower
Infrastructure
Education and Health
Law and Government
Energy
Agriculture and Forestry
Finance
Climate and Environment
Humanitarian Response
Water and Sanitation
Industry and Trade
Mining
Technical Cooperation
Transport```

In [28]:
def clean(x):
    try:
        return x.lower().strip()
    except:
        return 'NA'

In [29]:
sector_data = labels[labels.Sectors.notnull()].copy(deep=True)
sector_data.loc[:,'cl_Sector'] = sector_data.Sectors.apply(clean)

sector_dict={0:'MISC','not a project':'MISC','NA':'MISC'}
for k in sector_dict:
    sector_data.loc[sector_data.cl_Sector == k, 'cl_Sector'] = sector_dict[k]


In [30]:
sector_data[['Sectors','cl_Sector']].head()

,Sectors,cl_Sector
0,Infrastructure,infrastructure
1,Energy,energy
3,Transport,transport
4,"Construction, Finance","construction, finance"
5,"Construction, Finance","construction, finance"


In [31]:
sector_data.cl_Sector.value_counts(dropna=False)

energy                                                84
finance                                               40
infrastructure                                        38
transport                                             29
water and sanitation                                  28
education and health                                  12
MISC                                                  11
industry and trade                                    10
communications                                         9
construction, finance                                  6
hydropower                                             4
water and sanitation, climate                          3
agriculture and forestry, industry and trade           3
finance, energy                                        3
infrastructure, transport                              2
agriculture and forestry                               2
transport, industry and trade                          2
finance, education and health  

In [32]:
sector_data['top_sector'] = [i.split(',')[0] for i in sector_data.cl_Sector]

In [33]:
sector_data.top_sector.value_counts()

energy                      87
finance                     45
infrastructure              42
transport                   33
water and sanitation        31
education and health        12
MISC                        11
industry and trade          10
communications               9
construction                 6
agriculture and forestry     5
hydropower                   4
humanitarian response        1
Name: top_sector, dtype: int64

----------

## Banks 

```
Asian Infrastructure Investment Bank (AIIB)
African Development Bank (AFDB)
Asian Development Bank (ADB)
Green Climate Fund (GCF)
New Development Bank (NDB)
European Investment Bank (EIB)
World Bank (WB)
International Finance Corporation (IFC)
Inter-American Investment Corporation (IIC)
Netherlands Development Finance Company (FMO)
Inter-American Development Bank (IADB)
Multilateral Investment Guarantee Agency (MIGA)
European Bank for Reconstruction and Development (EBRD)
```

In [34]:
banks_data = labels[labels.Bank1.notnull()]
banks_data.Bank1 = banks_data.Bank1.apply(clean)
banks_data.Bank2 = banks_data.Bank2.apply(clean)

In [35]:
Counter(banks_data.Bank1.tolist() + banks_data.Bank2.tolist())

Counter({'NA': 422,
         'adb': 210,
         'afdb': 81,
         'aiib': 49,
         'ebrd': 68,
         'eib': 15,
         'gcf': 1,
         'idb': 5,
         'ifc': 2,
         'indonesia': 1,
         'jica': 3,
         'pakistan': 1,
         'wb': 8})

---------

## Country 

In [36]:
country_data = labels[labels.Country1.notnull()]
country_data.is_copy = False
country_data.Country1 = country_data.Country1.apply(clean)
country_data.Country2 = country_data.Country2.apply(clean)

In [37]:
country_data.Country1.value_counts()

india                                                                                                       70
philippines                                                                                                 24
pakistan                                                                                                    20
turkey                                                                                                      20
bangladesh                                                                                                  19
nigeria                                                                                                     17
mongolia                                                                                                    10
egypt                                                                                                        9
sri lanka                                                                                                    9
l

In [38]:
country_data = country_data[country_data.Country1 != 'NA']
print(country_data.shape)

(350, 16)


In [39]:
country_data = country_data.drop('Country2',axis=1)

## Clean files 

In [40]:
drop_list = ['DK_ADMIN_TRANSFERRED_CSV','HyperLink']

In [59]:
output_files = {}
for df in [('banks',banks_data),( 'sectors',sector_data), ('countries',country_data),( 'projects',project_labeled_data)]:
    out_df = df[1].drop(drop_list, axis=1).copy()
    if df[0] == 'banks':
        out_df = out_df.drop(['Sectors','Country1','Country2','Matched','Projects(EWSProjectID)','EWS Project Name',
                             'EWS hyperlink'],axis=1)
    if df[0] == 'sectors':
        out_df = out_df.drop(['Bank1','Bank2','Country1','Country2','Matched','Projects(EWSProjectID)','EWS Project Name',
                             'EWS hyperlink'],axis=1)
    if df[0] == 'countries':
        out_df = out_df.drop(['Bank1','Bank2','Sectors','Matched','Projects(EWSProjectID)','EWS Project Name',
                             'EWS hyperlink'],axis=1)
    if df[0] == 'projects':
        out_df = out_df.drop(['Bank1','Bank2','Sectors','Country2','Country1'],axis=1)
        out_df.rename(columns={"Projects(EWSProjectID)":"ProjectNumber"},inplace=True) 

    output_files[df[0]] = out_df

    #drop admin cols etc 

In [49]:
print(output_files['banks'].shape)
output_files['banks'].head()

(433, 7)


,article_id,published,title,url,feed_label,Bank1,Bank2
0,10f9ed2,2018-01-11,ADB Provides Support for Three Infrastructure ...,http://moderndiplomacy.eu/2018/01/11/adb-provi...,NEWS ADB - All Streams,adb,NA
1,c0eece9b,2018-05-13,ADB Helps Inaugurate New Power Distribution Ne...,http://feedproxy.google.com/~r/adb_news/~3/2My...,NEWS ADB - All Streams,adb,NA
3,d1d79dd8,2018-02-20,ADB Provides $360 Million for Rolling Stock to...,http://feedproxy.google.com/~r/adb_news/~3/v9s...,NEWS ADB - All Streams,adb,NA
4,f0d65e5,2018-02-25,ADB provides financing to Thailand's B.Grimm P...,https://www.dealstreetasia.com/stories/adb-b-g...,NEWS ADB - All Streams,adb,NA
5,4a557358,2018-02-26,ADB's $235m loan to support B.Grimm Power expa...,https://www.power-technology.com/news/adbs-235...,NEWS ADB - All Streams,adb,NA


In [50]:
print(output_files['sectors'].shape)
output_files['sectors'].head()

(296, 8)


,article_id,published,title,url,feed_label,Sectors,cl_Sector,top_sector
0,10f9ed2,2018-01-11,ADB Provides Support for Three Infrastructure ...,http://moderndiplomacy.eu/2018/01/11/adb-provi...,NEWS ADB - All Streams,Infrastructure,infrastructure,infrastructure
1,c0eece9b,2018-05-13,ADB Helps Inaugurate New Power Distribution Ne...,http://feedproxy.google.com/~r/adb_news/~3/2My...,NEWS ADB - All Streams,Energy,energy,energy
3,d1d79dd8,2018-02-20,ADB Provides $360 Million for Rolling Stock to...,http://feedproxy.google.com/~r/adb_news/~3/v9s...,NEWS ADB - All Streams,Transport,transport,transport
4,f0d65e5,2018-02-25,ADB provides financing to Thailand's B.Grimm P...,https://www.dealstreetasia.com/stories/adb-b-g...,NEWS ADB - All Streams,"Construction, Finance","construction, finance",construction
5,4a557358,2018-02-26,ADB's $235m loan to support B.Grimm Power expa...,https://www.power-technology.com/news/adbs-235...,NEWS ADB - All Streams,"Construction, Finance","construction, finance",construction


In [51]:
print(output_files['countries'].shape)

output_files['countries'].head()

(350, 6)


,article_id,published,title,url,feed_label,Country1
0,10f9ed2,2018-01-11,ADB Provides Support for Three Infrastructure ...,http://moderndiplomacy.eu/2018/01/11/adb-provi...,NEWS ADB - All Streams,cambodia
1,c0eece9b,2018-05-13,ADB Helps Inaugurate New Power Distribution Ne...,http://feedproxy.google.com/~r/adb_news/~3/2My...,NEWS ADB - All Streams,afghanistan
3,d1d79dd8,2018-02-20,ADB Provides $360 Million for Rolling Stock to...,http://feedproxy.google.com/~r/adb_news/~3/v9s...,NEWS ADB - All Streams,bangladesh
4,f0d65e5,2018-02-25,ADB provides financing to Thailand's B.Grimm P...,https://www.dealstreetasia.com/stories/adb-b-g...,NEWS ADB - All Streams,thailand
5,4a557358,2018-02-26,ADB's $235m loan to support B.Grimm Power expa...,https://www.power-technology.com/news/adbs-235...,NEWS ADB - All Streams,thailand


In [61]:
print(output_files['projects'].shape)

output_files['projects'].head()

(112, 9)


,article_id,published,title,url,feed_label,ProjectNumber,EWS Project Name,EWS hyperlink,Matched
0,10f9ed2,2018-01-11,ADB Provides Support for Three Infrastructure ...,http://moderndiplomacy.eu/2018/01/11/adb-provi...,NEWS ADB - All Streams,"ADB-41123-015, ADB-48158-001, ADB-41435-053",Road Network Improvement Project (formerly Sec...,https://ewsdata.rightsindevelopment.org/projec...,1.0
1,c0eece9b,2018-05-13,ADB Helps Inaugurate New Power Distribution Ne...,http://feedproxy.google.com/~r/adb_news/~3/2My...,NEWS ADB - All Streams,ADB-47282-001,Energy Supply Improvement Investment Program (...,https://ewsdata.rightsindevelopment.org/projec...,1.0
3,d1d79dd8,2018-02-20,ADB Provides $360 Million for Rolling Stock to...,http://feedproxy.google.com/~r/adb_news/~3/v9s...,NEWS ADB - All Streams,ADB-50312-003,Railway Rolling Stock Operations Improvement P...,https://ewsdata.rightsindevelopment.org/projec...,1.0
4,f0d65e5,2018-02-25,ADB provides financing to Thailand's B.Grimm P...,https://www.dealstreetasia.com/stories/adb-b-g...,NEWS ADB - All Streams,ADB-50410-001,ASEAN Distributed Power Project: Initial Pover...,https://ewsdata.rightsindevelopment.org/projec...,1.0
5,4a557358,2018-02-26,ADB's $235m loan to support B.Grimm Power expa...,https://www.power-technology.com/news/adbs-235...,NEWS ADB - All Streams,ADB-50410-001,ASEAN Distributed Power Project: Initial Pover...,https://ewsdata.rightsindevelopment.org/projec...,1.0


In [47]:
for k in output_files:
    output_files[k].to_csv('../../Data/Labeled_Data/'+k+'.csv',index=False)

-----------------

## Sanity Check 

In [98]:
idx = 40
test = output_files['projects'].iloc[idx]
test

article_id                                                   11e77947
published                                         2018-04-26 00:00:00
title               EBRD declares effective 218 mln euro loan for ...
url                 https://seenews.com/news/ebrd-declares-effecti...
feed_label                                    NEWS EBRD - All streams
ProjectNumber                                              EBRD-48132
EWS Project Name              KESH Restructuring Project (EBRD-48132)
EWS hyperlink       https://ewsdata.rightsindevelopment.org/projec...
Matched                                                             1
Name: 51, dtype: object

In [99]:
feedly[feedly.article_id == test.article_id ]

,article_id,title,url,keep,feed_label,content,published,summary,scraped_content,article_text,article_keywords,article_text_len,top_lang
81,11e77947,EBRD declares effective 218 mln euro loan for ...,https://seenews.com/news/ebrd-declares-effecti...,True,NEWS EBRD - All streams,NaN,2018-04-26 06:14:54,"<table border=""0"" cellspacing=""3"" cellpadding=...",(EBRD declares effective 218 mln euro loan for...,EBRD declares effective 218 mln euro loan for ...,"[ebrd, declares, 218, sector, mln, albanias, m...",1994,en


In [100]:
projects[projects.ProjectNumber == test.ProjectNumber]

,EWS ID,ProjectNumber,Published,Bank Risk Rating,Project Status,EWS URL,Detailed Analysis URL,Project Name,City,Country Count,...,Sector 7,Last Edited,Date Scraped,Date Disclosed,Board Date,Source URL,Project Cost,Investment Amount,Project Description,Contact Information
1286,19153,EBRD-48132,Published,B,Approved,https://ews.rightsindevelopment.org/projects/4...,NaN,KESH Restructuring Project,NaN,1.0,...,NaN,8/10/17,1/13/17,3/22/16,6/22/16,http://www.ebrd.com/work-with-us/projects/psd/...,246.02,246.02,This project finances a comprehensive program ...,"Ani Spahiu, Taulant Hoxha\r\rspahiua@kesh.al, ..."


--------------